<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-and-Load-Data" data-toc-modified-id="Setup-and-Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup and Load Data</a></span></li><li><span><a href="#Defender-Distance" data-toc-modified-id="Defender-Distance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defender Distance</a></span></li><li><span><a href="#Shot-Distance" data-toc-modified-id="Shot-Distance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Shot Distance</a></span></li><li><span><a href="#Shot-Distance-and-Defender-Distance" data-toc-modified-id="Shot-Distance-and-Defender-Distance-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Shot Distance and Defender Distance</a></span><ul class="toc-item"><li><span><a href="#Hexbin-Plot" data-toc-modified-id="Hexbin-Plot-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Hexbin Plot</a></span></li></ul></li><li><span><a href="#Expected-Shot-Value" data-toc-modified-id="Expected-Shot-Value-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Expected Shot Value</a></span></li><li><span><a href="#An-Improved-Shooting-Metric?" data-toc-modified-id="An-Improved-Shooting-Metric?-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>An Improved Shooting Metric?</a></span><ul class="toc-item"><li><span><a href="#Player-ESV" data-toc-modified-id="Player-ESV-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Player ESV</a></span></li><li><span><a href="#Team-ESV-per-85-FGA" data-toc-modified-id="Team-ESV-per-85-FGA-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Team ESV per 85 FGA</a></span></li><li><span><a href="#Player-Points-Above-Average-per-FGA" data-toc-modified-id="Player-Points-Above-Average-per-FGA-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Player Points Above Average per FGA</a></span></li><li><span><a href="#Team-Points-Above-Average-per-85-FGA" data-toc-modified-id="Team-Points-Above-Average-per-85-FGA-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Team Points Above Average per 85 FGA</a></span></li></ul></li><li><span><a href="#Does-Defender-Distance-Relate-to-Shooter-Quality?" data-toc-modified-id="Does-Defender-Distance-Relate-to-Shooter-Quality?-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Does Defender Distance Relate to Shooter Quality?</a></span></li></ul></div>

# Expected Value Modeling for NBA Shot Locations

We should expect that two important drivers of the likelihood of making a shot are the shot distance and how near the closest defender is.  Using data obtained from NBA.com for the 2014-2015 season, this demo explores how you can build an expected value model for shooting.

## Setup and Load Data

In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
%matplotlib inline
import datascience as ds
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from datascience_utils import hexbin_plot, sorted_boxplot
from datascience_topic import build_expected_shot_values_from_hexbin

In [ ]:
t = ds.Table.read_table('shot_logs_2014_15.csv.gz', sep=',')

In [ ]:
t.show(5)

## Defender Distance
There are lots of things you could do with this data but let's just jump right in with defender distance.

We use the 1ft buckets in `CLOSE_DEF_DIST_ROUNDED` to compute FG% as a function of defender distance.

After grouping, we plot the relation and... wait, why does defender distance not seem to matter!?

_Question_
+ What might explain this seeming lack of relationship?  Does shot type matter?  Anything else?

In [ ]:
result = t.select('CLOSE_DEF_DIST_ROUNDED', 'SHOT_RESULT_BIN').\
    group('CLOSE_DEF_DIST_ROUNDED', collect=np.mean)

result = result.relabel('SHOT_RESULT_BIN mean', 'SHOT_PCT')
result.where('CLOSE_DEF_DIST_ROUNDED', ds.are.below_or_equal_to(14)).\
    plot(column_for_xticks='CLOSE_DEF_DIST_ROUNDED', select='SHOT_PCT')

plt.gca().set_ylim(0, .7);

Okay, now we separate by shot type.  Clearly defender distance is a big deal for 3pt shots.  But it still doesn't seem that important for 2pt shots.

_Question_
+ Should 2pt shooting % be immune defender distance?  Why or why not?  Is there something else missing?

In [ ]:
result = t.select('PTS_TYPE', 'CLOSE_DEF_DIST_ROUNDED', 'SHOT_RESULT_BIN').\
    group(['PTS_TYPE', 'CLOSE_DEF_DIST_ROUNDED'], collect=np.mean)

result = result.relabel('SHOT_RESULT_BIN mean', 'SHOT_PCT')

result = result.pivot('PTS_TYPE', 'CLOSE_DEF_DIST_ROUNDED', values='SHOT_PCT', collect=np.mean)
result.where('CLOSE_DEF_DIST_ROUNDED', ds.are.below_or_equal_to(14)).\
    plot(column_for_xticks='CLOSE_DEF_DIST_ROUNDED', select=['2', '3'])

## Shot Distance

We can quickly visualize the variability in shot distance with a pair of histogram plots.  The distance of the NBA 3pt line is not uniform but generally you see the overall pattern of shots.

In [ ]:
t.select('PTS_TYPE', 'SHOT_DIST').\
    pivot('PTS_TYPE', 'SHOT_DIST').\
    bar('SHOT_DIST', select=['2', '3'])

We can group by shot type and shot distance (rounded to nearest half foot) to get a feel for how shooting percentage varies with shot distance.  Obviously close in shots are layups/dunks that are almost always converted.

In [ ]:
result = t.select('PTS_TYPE', 'SHOT_DIST_ROUNDED', 'SHOT_RESULT_BIN').\
    group(['PTS_TYPE', 'SHOT_DIST_ROUNDED'], collect=np.mean)

result = result.relabel('SHOT_RESULT_BIN mean', 'SHOT_PCT')

result = result.pivot('PTS_TYPE', 'SHOT_DIST_ROUNDED', values='SHOT_PCT', collect=np.mean)

result['2'][result['SHOT_DIST_ROUNDED'] > 24] = np.nan
result['3'][(result['SHOT_DIST_ROUNDED'] < 21) | (result['SHOT_DIST_ROUNDED'] > 29)] = np.nan

result.plot(column_for_xticks='SHOT_DIST_ROUNDED', select=['2', '3'])
plt.gca().set_ylim(0, .7);

## Shot Distance and Defender Distance

Okay, we saw defender distance mattered for 3s and shot distance matters.  Let's combine the two.  We use the shot distance buckets in `SHOT_DIST_BUCKET` as well as the rounded defender distance to show how shooting percentage varies both in shot distance and defender distance.

In [ ]:
result = t.select('SHOT_DIST_BUCKET', 'CLOSE_DEF_DIST_ROUNDED', 'SHOT_RESULT_BIN').\
    group(['SHOT_DIST_BUCKET', 'CLOSE_DEF_DIST_ROUNDED'], collect=np.mean)
result = result.relabel('SHOT_RESULT_BIN mean', 'SHOT_PCT')

result = result.pivot(
    'SHOT_DIST_BUCKET', 'CLOSE_DEF_DIST_ROUNDED', values='SHOT_PCT', collect=np.mean)
result.where('CLOSE_DEF_DIST_ROUNDED', ds.are.below_or_equal_to(14)).\
    plot(column_for_xticks='CLOSE_DEF_DIST_ROUNDED')
plt.gca().set_ylim(0, 1.03);

### Hexbin Plot

We can view the relationship between the in more continuous space with a hexbin plot.  A hexbin plot segments the space and all shots that lie within a hexagon will be grouped together.  Then within that hexagon, we will compute the shooting percentage.  We can see how regardless of shot distance, the defender distance near 0 drives down shooting percentage at that distance from the basket.

_Question_
+ Why are there some spots that are extra dark or white that don't really fit with the general pattern?
+ What is going on with that spike on the left side?  What kind of shot has a high percentage, is near the basket, and has no defender nearby?

In [ ]:
hexbin_plot(
    t.where('SHOT_DIST', ds.are.below_or_equal_to(30)),
    'SHOT_DIST',
    'CLOSE_DEF_DIST',
    C='SHOT_RESULT_BIN',
    collect=np.mean,
    gridsize=20,
    figsize=(10, 6),
    cmap=plt.cm.viridis_r,
    mincnt=5,
    vmin=0.2,
    vmax=0.8
);

## Expected Shot Value

We can do the same hexbin plot but compute expected points.  This makes more sense due to the 3pt line and represents a shooting efficiency due to the variable value of shots.

_Question_
+ What does this hexbin plot say about mid/long-distance 2s as well?  How bad is a closely guarded mid/long 2 point shot?

In [ ]:
hexbin_esv_plot = hexbin_plot(
    t.where('SHOT_DIST', ds.are.below_or_equal_to(30)),
    'SHOT_DIST',
    'CLOSE_DEF_DIST',
    C='PTS_MADE',
    collect=np.mean,
    gridsize=20,
    cmap=plt.cm.viridis_r,
    figsize=(10, 6),
    mincnt=5,
    vmin=0.5,
    vmax=1.6
)

## An Improved Shooting Metric?

Recall the Effective Field Goal Pct was given by
$$
    \text{eFG\%} = \frac{\mathit{FG} + .5 \cdot \mathit{3FG}}{\mathit{FGA}} = \frac{\text{Total Points (excluding FT)}}{2 \cdot \mathit{FGA}}
$$

Ignoring the division by 2, our ESV computation is akin to the EFG computation: expected points scored per attempt.

Here's a metric we can build along what we've seen already in baseball: for each shot use the hexbin plot to compute the ESV.  

$$
    \text{eSV} = \frac{\text{Total Expected Points}}{\mathit{FGA}}
$$

We can also compute the points above average for each shot as
$$
    \text{Points Above Average} = \text{Points Made} - \text{eSV}
$$

This is actually akin to what was proposed in this [paper][1] at the Sloan conference.

[1]: http://www.sloansportsconference.com/wp-content/uploads/2014/02/2014-SSAC-Quantifying-Shot-Quality-in-the-NBA.pdf

In [ ]:
# Use a helper function to get the bin (and therefore ESV) for each shot
esv = build_expected_shot_values_from_hexbin(
    t['SHOT_DIST'], t['CLOSE_DEF_DIST'], hexbin_esv_plot)
t['EXPECTED_SHOT_VALUE'] = esv

# remove Nan
not_nan = ~np.isnan(esv)
esv_t = t.where(not_nan)

In [ ]:
esv_t.show(5)

In [ ]:
esv_t.hist('EXPECTED_SHOT_VALUE', bins=50)

In [ ]:
esv_t['PTS_ABOVE_AVG'] = esv_t['PTS_MADE'] - esv_t['EXPECTED_SHOT_VALUE']
esv_t.hist('PTS_ABOVE_AVG', bins=50)

### Player ESV
Which players are taking the highest value shots?  How about the lowest?

In [ ]:
result = esv_t.select('playerName', 'EXPECTED_SHOT_VALUE').\
    group('playerName', collect=np.mean)
result_ct = esv_t.select('playerName').group('playerName')

player_esv = result.join('playerName', result_ct)

player_esv = player_esv.relabel('EXPECTED_SHOT_VALUE mean', 'ESVperFGA').\
    relabel('count', 'FGA')

In [ ]:
player_esv.where('FGA', ds.are.above(100)).\
    sort('ESVperFGA', descending=True).\
    show(20)

In [ ]:
player_esv.where('FGA', ds.are.above(100)).\
    sort('ESVperFGA', descending=False).\
    show(20)

### Team ESV per 85 FGA 

Which teams take the best shots?  85 FGA is about 1 game.

In [ ]:
result = esv_t.select('PLAYER_TEAM', 'EXPECTED_SHOT_VALUE').\
    group('PLAYER_TEAM', collect=np.mean)
result_ct = esv_t.select('PLAYER_TEAM').group('PLAYER_TEAM')

team_esv = result.join('PLAYER_TEAM', result_ct)

team_esv = team_esv.relabel('EXPECTED_SHOT_VALUE mean', 'ESVperFGA').\
    relabel('count', 'FGA')
    
team_esv['ESVper85FGA'] = 85 * team_esv['ESVperFGA']
team_esv.sort('ESVper85FGA', descending=True).show()

### Player Points Above Average per FGA
We can compute PAA per FGA for each player.

In [ ]:
result = esv_t.select('playerName', 'PTS_ABOVE_AVG').\
    group('playerName', collect=np.mean)
result_ct = esv_t.select('playerName').group('playerName')   

paa = result.join('playerName', result_ct)

paa.relabel('PTS_ABOVE_AVG mean', 'PAAperFGA').\
    relabel('count', 'FGA')

In [ ]:
paa.where('FGA', ds.are.above(100)).\
    sort('PAAperFGA', descending=True).\
    show(20)

In [ ]:
paa.where('FGA', ds.are.above(100)).\
    sort('PAAperFGA', descending=True).\
    take[-20:].show()

### Team Points Above Average per 85 FGA 
We can compute PAA per 85 FGA for each team.

In [ ]:
paa_team = esv_t.select('PLAYER_TEAM', 'PTS_ABOVE_AVG').\
    group('PLAYER_TEAM', collect=np.mean).\
    relabel('PTS_ABOVE_AVG mean', 'PAAperFGA')
paa_team['PAAper85FGA'] = 85 * paa_team['PAAperFGA']
paa_team.sort('PAAper85FGA', descending=True).show()

## Does Defender Distance Relate to Shooter Quality?

Presumably better shooters will be guarded more closely, right?  To study this, let's restrict to 3pt shots.

In [ ]:
t.where('playerName', 'Stephen Curry').\
    where('PTS_TYPE', 3).\
    hist('CLOSE_DEF_DIST', bins=30)

We take all players with at least 50 shots, sort them by their 3FG% (from left to right), and construct boxplots showing defender distance.  If 3FG% indicates better shooters, and it should, then those players should be guarded tighter than poor shooters. We should see some kind of pattern in the plot that will reflect this hypothesis.

_Question_
+ What pattern emerges from this plot?  Can you think of a plausible explanation? 

In [ ]:
num_3pt_shots = 50
by = 'playerName'
column = "CLOSE_DEF_DIST"

three_pt_shots = t.where('PTS_TYPE', 3)

result = three_pt_shots.select(by, 'SHOT_RESULT_BIN').\
    group(by, collect=np.mean)
result2 = three_pt_shots.select(by).group(by)
result = result.join(by, result2)
result.relabel('SHOT_RESULT_BIN mean', '3FG%').\
    relabel('count', 'FGA')
    
by_3pt_pct = result.where('FGA', ds.are.above(num_3pt_shots)).\
    sort('3FG%', descending=True) 
    
sorted_boxplot(
    three_pt_shots.to_df(),
    by,
    column,
    by_3pt_pct[by],
    figsize=(25, 10),
    fontsize=8
)